## CC3092 - Deep Learning y Sistemas Inteligentes
## Deep Learning y Sistemas Inteligentes
## - Hoja de Trabajo 2 -

## Integrantes: 

 - Christopher García
 - Alejandro Gómez
 - Gabriel Vicente

## Instrucciones:
- Esta es una actividad en grupos de 3 personas máximo
- No se permitirá ni se aceptará cualquier indicio de copia. De presentarse, se procederá según el reglamento
correspondiente.
- Tendrán hasta el día indicado en Canvas.
Ejercicio 1 - Experimentación Práctica
En esta actividad, implementará y comparará diferentes funciones de pérdida y técnicas de regularización
utilizando PyTorch. Utilizará el conjunto de datos de Iris para una tarea de clasificación y una arquitectura básica de
red neuronal de feedforward. El objetivo es observar cómo las diferentes opciones impactan la convergencia y el
rendimiento del modelo.

# Ejercicio 1 - Experimentación Práctica

En esta actividad, implementará y comparará diferentes funciones de pérdida y técnicas de regularización
utilizando PyTorch. Utilizará el conjunto de datos de Iris para una tarea de clasificación y una arquitectura básica de
red neuronal de feedforward. El objetivo es observar cómo las diferentes opciones impactan la convergencia y el
rendimiento del modelo.

### Task 1 - Preparación del conjunto de datos

Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos
en conjuntos de entrenamiento y validación.

In [1]:
""" Librerias necesarias """

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset


In [2]:
""" Load data """

iris = load_iris()
X = iris.data
y = iris.target 

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



Tamaño del conjunto de entrenamiento: (120, 4)
Tamaño del conjunto de validación: (30, 4)


### Task 2 - Arquitectura modelo
Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas
ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa.

In [3]:

""" RN simple usando pytorch modules """

import torch
import torch.nn as nn

class SimpleFeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(SimpleFeedforwardNN, self).__init__()

        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        

        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_sizes) - 1):
            self.hidden_layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))

        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)
        

        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.activation(self.input_layer(x))
        for layer in self.hidden_layers:
            x = self.activation(layer(x))
        x = self.output_layer(x)
        return x

input_size = X_train.shape[1]
hidden_sizes = [64, 32]
output_size = 3

model = SimpleFeedforwardNN(input_size, hidden_sizes, output_size)
print(model)




SimpleFeedforwardNN(
  (input_layer): Linear(in_features=4, out_features=64, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=64, out_features=32, bias=True)
  )
  (output_layer): Linear(in_features=32, out_features=3, bias=True)
  (activation): ReLU()
)


### Task 3 - Funciones de Pérdida

Utilice diferentes funciones de pérdida comunes como Cross-Entropy Loss y MSE para clasificación. Entrene el
modelo con diferentes funciones de pérdida y registre las pérdidas de entrenamiento y test. Debe utilizar al menos 3
diferentes funciones. Es decir, procure que su código sea capaz de parametrizar el uso de diferentes funciones de
pérdida.

In [4]:

""" Entrenamiento, funcion de perdida y registro """

def train_model(model, loss_fn, optimizer, num_epochs=50):
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)
        
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    return train_losses, val_losses

input_size = X_train.shape[1]
hidden_sizes = [64, 32]
output_size = 3
num_epochs = 50
learning_rate = 0.001
model = SimpleFeedforwardNN(input_size, hidden_sizes, output_size)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_model(model, loss_fn, optimizer, num_epochs)

Epoch [1/50], Train Loss: 1.0347, Val Loss: 0.9240
Epoch [2/50], Train Loss: 0.8938, Val Loss: 0.8406
Epoch [3/50], Train Loss: 0.8199, Val Loss: 0.7661
Epoch [4/50], Train Loss: 0.7468, Val Loss: 0.6868
Epoch [5/50], Train Loss: 0.6701, Val Loss: 0.6275
Epoch [6/50], Train Loss: 0.6059, Val Loss: 0.5569
Epoch [7/50], Train Loss: 0.5415, Val Loss: 0.5077
Epoch [8/50], Train Loss: 0.4961, Val Loss: 0.4697
Epoch [9/50], Train Loss: 0.4606, Val Loss: 0.4361
Epoch [10/50], Train Loss: 0.4320, Val Loss: 0.4163
Epoch [11/50], Train Loss: 0.4094, Val Loss: 0.3943
Epoch [12/50], Train Loss: 0.3896, Val Loss: 0.3751
Epoch [13/50], Train Loss: 0.3792, Val Loss: 0.3561
Epoch [14/50], Train Loss: 0.3570, Val Loss: 0.3505
Epoch [15/50], Train Loss: 0.3371, Val Loss: 0.3247
Epoch [16/50], Train Loss: 0.3220, Val Loss: 0.3146
Epoch [17/50], Train Loss: 0.3034, Val Loss: 0.3020
Epoch [18/50], Train Loss: 0.2872, Val Loss: 0.2829
Epoch [19/50], Train Loss: 0.2713, Val Loss: 0.2766
Epoch [20/50], Train 

([1.0346800009409587,
  0.8937852541605632,
  0.8198907891909282,
  0.7468416770299275,
  0.670071562131246,
  0.6059257427851359,
  0.5414753715197246,
  0.49612850149472554,
  0.4606103777885437,
  0.43200918237368263,
  0.40941513180732725,
  0.3895662784576416,
  0.379242742061615,
  0.3570469816525777,
  0.33709455529848736,
  0.32198080817858377,
  0.30337822834650674,
  0.2872300366560618,
  0.2713353981574376,
  0.2557269970575968,
  0.2411670724550883,
  0.22946160236994426,
  0.21359724700450897,
  0.19912118911743165,
  0.18915411904454232,
  0.17871310114860534,
  0.17467562158902486,
  0.16050993502140046,
  0.1552472561597824,
  0.15252178435524305,
  0.1384110152721405,
  0.1341510981321335,
  0.12973794639110564,
  0.13367379754781722,
  0.12077055325110754,
  0.11770466963450114,
  0.11314948250850042,
  0.11766246172289054,
  0.11233008652925491,
  0.10095376918713252,
  0.10912240991989772,
  0.10109676023324331,
  0.10383709371089936,
  0.09624977658192317,
  0.0954